In [ ]:
%reload_ext openad.notebooks.styles

<!-- Header banner -->
<div class="banner"><div>OpenAD Demonstration</div><b>OpenAD<span>Tutorial</span></b></div>

## OpenAD Magic Commands

When using Magic commands to access the Openad toolkit you have 2 options 

1. `%openad` provides a simple user interface that provides styled and formatted objects back to the notebook. Tables use pandas Dataframe Styler object. These can be converted back to data frame objects using `.data` on the object or using the in memory assistant which will copy the last result to a file , dataframe or to the dataviewer.
  When this is available you will see `Next up, you can run: result open/edit/copy/display/as dataframe/save [as '<filename.csv>']` in the output.
  
  This magic command is the recommended version to use as it will provide all warning and results visually.
  
2. `%openadd` is the second form that allows you to return api style results in dataframe or list formats that can be used programatically for functions or flows in your notebook. This is good for prebuilt notebook process flows.

### Available Commands

To learn what commands available you simply need to enter a `?` at the command line or at a magic command to list avialable commands

In [ ]:
%openad ?

## Plugin Help

To look at the help purely for a specific toolkit, you simply need to ensure the toolkit is added then run the following.

%openad add toolkit rxn

In [ ]:
%openad ? rxn

## Working with OpenAD

There are a Range of different commands that can be run to navigate through the toolkit, looking at `workspaces`, `toolkits`, `files` and `runs` within `workspaces` etc.<br> For example:

In [ ]:
%openad list workspaces

In [ ]:
%openad list files 

### Runs

The concept of a run is to be able to re execute a consecutive set of commands in order, ideally that provide information to one another.

In [ ]:
%openad ? run

We are now going to create a run for demostration purposes.

In [ ]:
%openad create run

In [ ]:
%openad list files

In [ ]:
%openad rxn list models

In [ ]:
%openad list runs

When we want to stop and record the run we simply execute the following.

In [ ]:
%openad save run as test_run3

We can display its contents with the following command.

In [ ]:
%openad display run test_run3

Now let's run it.

In [ ]:
%openad run test_run3

## Demonstration: Deep Search to RXN

We'll use <b>Deep Search</b> to scan patents for mentions of a drug, then use <b>RXN</b> to determine retrosynthesis paths.

First, let's ask the AI assistant how we should do it.

In [ ]:
mol = %openadd @aspirin>>canonical_smiles

%openad ds search for patents containing molecule '{mol}'

%openad rxn predict retrosynthesis '{mol}'

## Demonstration: Sequence of commands

#### 1. Get a molecule's SMILES

In [ ]:
target_smile = %openadd @tritace>>canonical_smiles

#### 2. Query Deep Search for patent mentions

In [ ]:
%openad ds search for patents containing molecule '{target_smile}'

#### 3. Inspect available domains and collections at Deep Search

In [ ]:
%openad ds display all collections 

#### 4. Learn about the collection we may find the most interesting

In [ ]:
%openad ds display collection details 'Crossref'

#### 5. Find which collections mention the molecule 'Sibofimloc'

In [ ]:
%openad ds display collection matches for 'Sibofimloc'

#### 6. Find papers discussing Sibofimloc and Safety

In [ ]:
%openad ds search collection 'crossref' for 'Sibofimloc AND safety' show (docs data)

#### 7. Inspect molecule properties

In [ ]:
%openad show molecule Sibofimloc

#### 8. Find similar molecules

This will let us consider similar targets to pursue if our further experiments don't work as intended.

In [ ]:
target_string = %openadd @Sibofimloc>>canonical_smiles
similar_targets = %openadd ds search for similar molecules to '{target_string}' save as 'similar_targets.csv'
display(similar_targets)

#### 9. Visually inspect molecules

In [ ]:
%openad load molecules using dataframe similar_targets
%openad show molecules 

#### 10. Find patents related to target

In [ ]:
%openad ds search for patents containing molecule '{target_string}'

#### 11. Find patents related to similar targets

In [ ]:
import pandas
patent_list = []
for similar_mol in similar_targets['smiles']:
    results_df = %openadd ds search for patents containing molecule '{similar_mol}'
    patent_list = patent_list + results_df.to_dict(orient="records")

patent_list_df = pandas.DataFrame(patent_list)
patent_list_df

#### 12. Gather molecules from related patents

In [ ]:
%openad ds search for molecules in patents from dataframe patent_list_df

#### 13. Visualize all results

In [ ]:
%openad result open

#### 14. Visualize individual result

In [ ]:
%openad display molecule CCC(C)C(=O)N1CCC2(CC1)c1cc(C#CC3OC(CO)C(O)C(O)C3O)ccc1-c1ccc(C#CC3OC(CO)C(O)C(O)C3O)cc12

## Retrosynthesis Prediction

We'll run the retrosynthesis prediction on our original target molecule `Sibofimloc` to determine how to snythesize the molecule.

In [ ]:
target_string = 'CCC(C)C(=O)N1CCC2(CC1)c1cc(C#CC3OC(CO)C(O)C(O)C3O)ccc1-c1ccc(C#CC3OC(CO)C(O)C(O)C3O)cc12'
%openad rxn predict retrosynthesis '{target_string}' using( max_steps=3 )

In [ ]:
%openad add molecule {target_string}
%openad enrich molecules with analysis
%openad show molecule {target_string}

## Interpreting  Recipes

In [ ]:
%openad rxn interpret recipe 'To a stirred solution of 7-(difluoromethylsulfonyl)-4-fluoro-indan-1-one (110 mg, 0.42 mmol)\
in methanol (4 mL) was added sodium borohydride \
(24 mg, 0.62 mmol). The reaction mixture was stirred at ambient temperature for 1 hour.'

## Predicting Reaction Outcomes

#### Single Prediction

In [ ]:
%openad rxn predict reaction 'Cl.CCC(=O)NCCC.O' use_saved

In [ ]:
%openad import from '~/openad_notebooks/sample_files/reactions.csv' to 'reactions.csv'
%openad rxn predict reaction in batch from file 'reactions.csv'

#### Batch Predictions

In [ ]:
%openad rxn predict reaction in batch from list ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'Cl.CCC(=O)NCCC.O'] 

#### Using the [use_saved] clause

Providing the ability to reuse recently run previous reaction runs

In [ ]:
%openad rxn predict reaction in batch  from list ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'BrBr.c1ccc2cc3ccccc3cc2c1','Cl.CCC(=O)NCCC.O'] use_saved

In [ ]:

import pandas
lst = ['BrBr.c1ccc2cc3ccccc3cc2c1CCO' ,'Cl.CCC(=O)NCCC.O'] 
batch_list = pandas.DataFrame(lst,columns=['reactions'])
%openad rxn predict reaction topn in batch from dataframe batch_list using (topn=3) use_saved

## Quickly investingation and adding to a set of molecules

In [ ]:
%openad add molecule tritace force
%openad add molecule aspirin force
%openad add molecule Ibuprofen force
%openad add molecule entresto force

In [ ]:
%openad list molecules

In [ ]:
mol = %openadd @tritace>>canonical_smiles
%openad set context rxn
%openad predict retrosynthesis '{mol}' using( max_steps=3 )

In [ ]:
%openad set context ds4sd
%openad search for patents containing molecule '{mol}'
%openad search for similar molecules to '{mol}'

In [ ]:
%openad list molecules

In [ ]:
%openad enrich molecules with analysis 
%openad display molecule tritace

In [ ]:
%openad list molecules

In [ ]:
%openad show molecules